In [4]:
!pip3 install requests geopandas geopy tweepy selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [5]:
# accessing APIs and URLs
import requests
import tweepy

# handling of JSON responses
import json
from pprint import pprint
from pandas import json_normalize

# general data handling
# note: you need to additionally install geopy
import geopandas as gpd 
import pandas as pd

# static web scraping
from lxml.html import parse, fromstring

# selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import (
    WebDriverWait)
from selenium.webdriver.support import (
    expected_conditions as EC)
from selenium.webdriver.common.by import By

import time

In [6]:
r = requests.get("https://www.googleapis.com/"
                 "books/v1/volumes?q=python")
data = r.json()
print(data.keys())  # 'items' seems most promising
pprint(data['items'][0]) # let's print the 1st one

dict_keys(['kind', 'totalItems', 'items'])
{'accessInfo': {'accessViewStatus': 'SAMPLE',
                'country': 'NL',
                'embeddable': True,
                'epub': {'isAvailable': True},
                'pdf': {'isAvailable': True},
                'publicDomain': False,
                'quoteSharingAllowed': False,
                'textToSpeechPermission': 'ALLOWED',
                'viewability': 'PARTIAL',
                'webReaderLink': 'http://play.google.com/books/reader?id=9MS9BQAAQBAJ&hl=&printsec=frontcover&source=gbs_api'},
 'etag': 'H+xcoOyMoDw',
 'id': '9MS9BQAAQBAJ',
 'kind': 'books#volume',
 'saleInfo': {'country': 'NL', 'isEbook': False, 'saleability': 'NOT_FOR_SALE'},
 'searchInfo': {'textSnippet': 'In Black Hat Python, the latest from Justin '
                               'Seitz (author of the best-selling Gray Hat '
                               'Python), you’ll explore the darker side of '
                               'Python’s capabilities—wr

In [7]:
d = json_normalize(data['items'])
d.head()

,kind,id,etag,selfLink,volumeInfo.title,volumeInfo.subtitle,volumeInfo.authors,volumeInfo.publisher,volumeInfo.publishedDate,volumeInfo.description,...,searchInfo.textSnippet,volumeInfo.averageRating,volumeInfo.ratingsCount,saleInfo.listPrice.amount,saleInfo.listPrice.currencyCode,saleInfo.retailPrice.amount,saleInfo.retailPrice.currencyCode,saleInfo.buyLink,saleInfo.offers,accessInfo.pdf.acsTokenLink
0,books#volume,9MS9BQAAQBAJ,H+xcoOyMoDw,https://www.googleapis.com/books/v1/volumes/9M...,Black Hat Python,Python Programming for Hackers and Pentesters,[Justin Seitz],No Starch Press,2014-12-14,"In Black Hat Python, the latest from Justin Se...",...,"In Black Hat Python, the latest from Justin Se...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,books#volume,4pgQfXQvekcC,N4NQ+G559nM,https://www.googleapis.com/books/v1/volumes/4p...,Learning Python,Powerful Object-Oriented Programming,[Mark Lutz],"""O'Reilly Media, Inc.""",2013-06-12,"Get a comprehensive, in-depth introduction to ...",...,"Get a comprehensive, in-depth introduction to ...",3.5,4.0,46.87,EUR,46.87,EUR,https://play.google.com/store/books/details?id...,"[{'finskyOfferType': 1, 'listPrice': {'amountI...",NaN
2,books#volume,2ZggjwEACAAJ,TfAWJPA0QiE,https://www.googleapis.com/books/v1/volumes/2Z...,The Hitchhiker's Guide to Python,Best Practices for Development,"[Kenneth Reitz, Tanya Schlusser]",NaN,2016-07-25,The Hitchhiker's Guide to Python takes the jou...,...,Ready to complete your trek from journeyman to...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,books#volume,UEu0CAAAQBAJ,KZl82RSEyus,https://www.googleapis.com/books/v1/volumes/UE...,Automate the Boring Stuff with Python,Practical Programming for Total Beginners,[Al Sweigart],No Starch Press,2015-04-14,If you’ve ever spent hours renaming files or u...,...,"Even if you’ve never written a line of code, y...",4.5,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,books#volume,BP_WAgAAQBAJ,/FcpN33EMbI,https://www.googleapis.com/books/v1/volumes/BP...,Learning Python with Raspberry Pi,NaN,"[Alex Bradbury, Russel Winder, Ben Everard]",John Wiley & Sons,2014-03-10,Explains how to leverage the revolutionary Ras...,...,This approachable book serves as an ideal reso...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
allitems = []
i = 0
while True:
    r = requests.get("https://www.googleapis.com/"
        "books/v1/volumes?q=python&maxResults="
        f"40&startIndex={i}")
    data = r.json()
    try:
        allitems.extend(data['items'])
    except:
        print(f"Retrieved {len(allitems)},"
              "it seems like that's it")
        break
    i+=40
d = json_normalize(allitems)

Retrieved 380,it seems like that's it


In [10]:
# first, go to https://developer.twitter.com/ and 
# create a so-called 'app'

api_key='COPY THIS FROM TWITTER'
api_secret = 'COPY THIS FROM TWITTER'

access_token = 'COPY THIS FROM TWITTER'
access_token_secret = 'COPY THIS FROM TWITTER'

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, 
                      access_token_secret)

api = tweepy.API(auth)

results = pd.DataFrame([x._json for x in 
                        api.search('#covid')])
results.head()

TweepError: [{'code': 89, 'message': 'Invalid or expired token.'}]

In [ ]:
gpd.tools.geocode(['Amsterdam','New York'])

In [ ]:
# first, go to https://developer.twitter.com/ and create a so-called 'app'

api_key='COPY THIS FROM TWITTER'
api_secret = 'COPY THIS FROM TWITTER'
access_token = 'COPY THIS FROM TWITTER'
access_token_secret = 'COPY THIS FROM TWITTER'

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)

replyusers = set(tweets.reply_to_screen_name) - set(tweets.screen_name)
replyusers_small = list(replyusers)[:20]

api = tweepy.API(auth)
result = api.lookup_users(screen_names=replyusers_small)

users = []
for user in result:
    if user._json['location'] =="":
        coordinates = None
    else:
        coordinates = gpd.tools.geocode(user._json['location'])
    users.append({'screen_name': user._json['screen_name'], 'location':user._json['location'], 'coordinates': coordinates})

pd.DataFrame(users).head()